In [1]:
from PIL import Image
import matplotlib.pyplot as plt

def pixelate(image, block_size):
    width, height = image.size
    num_blocks_x = width // block_size
    num_blocks_y = height // block_size

    # Resize the image to the desired size
    small_image = image.resize((num_blocks_x, num_blocks_y), resample=Image.BOX)
    # Scale the image back up to the original size
    pixelated_image = small_image.resize(image.size, resample=Image.NEAREST)

    return pixelated_image

# Load the image
image_path = 'B.png'
image = Image.open(image_path)

# Define the block size (adjust as needed)
block_size = 4

# Pixelate the image
pixelated_image = pixelate(image, block_size)
plt.imshow(pixelated_image, cmap="gray")

FileNotFoundError: [Errno 2] No such file or directory: 'B.png'

In [6]:
directory_path = "./numbers/Font"

In [9]:
import os
os.listdir(directory_path)

['alphabet', 'number']

In [25]:
def pixelate_image(image, block_size):
    width, height = image.size
    num_blocks_x = width // block_size
    num_blocks_y = height // block_size

    # Resize the image to the desired size
    small_image = image.resize((num_blocks_x, num_blocks_y), resample=Image.BOX)
    # Scale the image back up to the original size
    pixelated_image = small_image.resize(image.size, resample=Image.NEAREST)

    return pixelated_image

def pixelate_images_in_directory(directory, block_size=4):
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                image = Image.open(file_path)
                pixelated_image = pixelate_image(image, block_size)
                pixelated_image.save(file_path)
            except IOError:
                print(f"Unable to open or save image: {file_path}")

# Directory containing the subdirectories of images
main_directory = "./numbers/Font"

# Pixelation block size (adjust as needed)
block_size = 4

# Iterate through the subdirectories and pixelate images
for subdirectory in ['alphabet', 'number']:
    subdirectory_path = os.path.join(main_directory, subdirectory)
    if os.path.isdir(subdirectory_path):
        pixelate_images_in_directory(subdirectory_path, block_size)

In [24]:
os.path.isdir("./alphabet")

False

In [29]:
!pip install torchvision

Alphabet

In [42]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

# Hyperparameters
batch_size = 32
learning_rate = 0.001
num_epochs = 10

# Data preprocessing and augmentation
transform = transforms.Compose([
    transforms.Grayscale(),  # Convert to grayscale
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Load the dataset
dataset = ImageFolder('./numbers/Font/alphabet', transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define the convolutional neural network
class AlphabetClassifier(nn.Module):
    def __init__(self):
        super(AlphabetClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(32 * 32 * 32, 64)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(64, 26)  # 26 output classes for alphabets

    def quantize(self, a, total_bits = 32):
        integer_format = 1+torch.ceil(torch.log2(torch.abs(a+1)))*(a>=0) + torch.ceil(torch.log2(torch.abs(a)))*(a<0)
        fractional_format = total_bits - integer_format
        x = torch.div(torch.floor(a * 2**fractional_format), 2**fractional_format)
        return x

    def forward(self, x, quant=False):
        if(quant == True):
            x = self.quantize(x)
        # convolve
        x = self.conv1(x)
        # batchnorm
        x = self.bn1(x)
        # quantize
        if(quant == True):
            x = self.quantize(x)
        # relu
        x = self.relu1(x)
        # pool
        x = self.pool1(x)
        x = self.conv2(x)
        if(quant == True):
            x = self.quantize(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        if(quant == True):
            x = self.quantize(x)
        x = self.relu3(x)
        x = self.fc2(x)
        if(quant == True):
            x = self.quantize(x)
        return x

# Initialize the model
model = AlphabetClassifier()

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
total_step = len(dataloader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(dataloader):
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print training progress
        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_step}], Loss: {loss.item():.4f}')

# Save the model
torch.save(model.state_dict(), 'alphabet_classifier.pth')


Device: cpu
Epoch [1/10], Step [100/826], Loss: 0.5399
Epoch [1/10], Step [200/826], Loss: 0.7661
Epoch [1/10], Step [300/826], Loss: 0.5319
Epoch [1/10], Step [400/826], Loss: 0.2938
Epoch [1/10], Step [500/826], Loss: 0.0858
Epoch [1/10], Step [600/826], Loss: 0.2480
Epoch [1/10], Step [700/826], Loss: 0.0697
Epoch [1/10], Step [800/826], Loss: 0.1713
Epoch [2/10], Step [100/826], Loss: 0.0309
Epoch [2/10], Step [200/826], Loss: 0.0528
Epoch [2/10], Step [300/826], Loss: 0.0255
Epoch [2/10], Step [400/826], Loss: 0.6666
Epoch [2/10], Step [500/826], Loss: 0.0374
Epoch [2/10], Step [600/826], Loss: 0.4059
Epoch [2/10], Step [700/826], Loss: 0.1279
Epoch [2/10], Step [800/826], Loss: 0.0060
Epoch [3/10], Step [100/826], Loss: 0.0042
Epoch [3/10], Step [200/826], Loss: 0.0371
Epoch [3/10], Step [300/826], Loss: 0.0028
Epoch [3/10], Step [400/826], Loss: 0.0399
Epoch [3/10], Step [500/826], Loss: 0.0105
Epoch [3/10], Step [600/826], Loss: 0.0023
Epoch [3/10], Step [700/826], Loss: 0.0223

Number

In [64]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

# Hyperparameters
batch_size = 32
learning_rate = 0.001
num_epochs = 5

# Data preprocessing and augmentation
transform = transforms.Compose([
    transforms.Grayscale(),  # Convert to grayscale
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load the dataset
dataset = ImageFolder('./numbers/Font/number', transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define the convolutional neural network
class NumberClassifier(nn.Module):
    def __init__(self):
        super(NumberClassifier, self).__init__()
        self.bn1 = nn.BatchNorm2d(16)
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(32 * 32 * 32, 64)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(64, 10)  # 10 output classes for numbers

    def quantize(self, a, total_bits = 32):
        integer_format = 1+torch.ceil(torch.log2(torch.abs(a+1)))*(a>=0) + torch.ceil(torch.log2(torch.abs(a)))*(a<0)
        fractional_format = total_bits - integer_format
        x = torch.div(torch.floor(a * 2**fractional_format), 2**fractional_format)
        return x

    def forward(self, x, quant=False):
        if(quant == True):
            x = self.quantize(x)
        x = self.conv1(x)
        x = self.bn1(x)
        if(quant == True):
            x = self.quantize(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        if(quant == True):
            x = self.quantize(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        if(quant == True):
            x = self.quantize(x)
        x = self.relu3(x)
        x = self.fc2(x)
        if(quant == True):
            x = self.quantize(x)
        return x

# Initialize the model
model = NumberClassifier()

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
total_step = len(dataloader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(dataloader):
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print training progress
        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_step}], Loss: {loss.item():.4f}')

# Save the model
torch.save(model.state_dict(), 'number_classifier.pth')


Epoch [1/5], Step [100/318], Loss: 0.2594
Epoch [1/5], Step [200/318], Loss: 0.0396
Epoch [1/5], Step [300/318], Loss: 0.0113
Epoch [2/5], Step [100/318], Loss: 0.0100
Epoch [2/5], Step [200/318], Loss: 0.0360
Epoch [2/5], Step [300/318], Loss: 0.0128
Epoch [3/5], Step [100/318], Loss: 0.0088
Epoch [3/5], Step [200/318], Loss: 0.0026
Epoch [3/5], Step [300/318], Loss: 0.0030
Epoch [4/5], Step [100/318], Loss: 0.0029
Epoch [4/5], Step [200/318], Loss: 0.0016
Epoch [4/5], Step [300/318], Loss: 0.0119
Epoch [5/5], Step [100/318], Loss: 0.0001
Epoch [5/5], Step [200/318], Loss: 0.0000
Epoch [5/5], Step [300/318], Loss: 0.0000


In [36]:
import cv2
image = cv2.imread("./numbers/Font/alphabet/A/img011-00001.png")


image.shape

(128, 128, 3)

In [61]:
import socket

# Data to be sent
cameraId = "camera1"
detected = True
numberPlate = "ABC123"

# Format the data
requestData = {
    cameraId : "camera1",
    detected : True,
    numberPlate : "ABC123"
}

# Endpoint details
host = "https://codehackerone-friendly-space-lamp-xxw6vjr9xgq36g74-5100.preview.app.github.dev/"
port = 5100

# Create a socket
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

try:
    # Connect to the server
    client_socket.connect((host, port))

    # Send data to the server
    client_socket.sendall(requestData.encode())

    # Receive response from the server
    response = client_socket.recv(1024)
    print("Response:", response.decode())

except ConnectionRefusedError:
    print("Error: Connection refused. Make sure the server is running.")

finally:
    # Close the socket connection
    client_socket.close()


gaierror: [Errno 11001] getaddrinfo failed